## Mount Azure Data Lake using Service Principal
1. Get client_id, tenant_id and client_secret from key vault
2. Set Spark Config with App/ Client Id, Directory/ Tenant Id & Secret
3. Call file system utility mount to mount the storage
4. Explore other file system utilities related to mount (list all mounts, unmount)

In [0]:
def mount_adls(storage_account_name, container_name):
    # Get Secrets from Key Vault
    client_id = dbutils.secrets.get(scope="formula1-scope", key="formula1dl-client-id")
    tenant_id = dbutils.secrets.get(scope="formula1-scope", key="formula1dl-tenant-id")
    client_secret = dbutils.secrets.get(scope="formula1-scope", key="formula1dl-client-secret")

    # Set spark configurations
    configs = {"fs.azure.account.auth.type": "OAuth",
                "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
                "fs.azure.account.oauth2.client.id": client_id,
                "fs.azure.account.oauth2.client.secret": client_secret,
                "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
    
    #Unmount the mount point if it already exists
    if any(mount.mountPoint == f"/mnt/{storage_account_name}/{container_name}" for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")
        
    # Mount the storage account container
    dbutils.fs.mount(
        source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
        mount_point = f"/mnt/{storage_account_name}/{container_name}",
        extra_configs = configs)
    
    display(dbutils.fs.mounts())


## Mount Containers

In [0]:
mount_adls("formula1dlks7", "raw")

In [0]:
mount_adls("formula1dlks7", "processed")

In [0]:
mount_adls("formula1dlks7", "presentation")

In [0]:
mount_adls("formula1dlks7", "demo")